In [ ]:
"""
48-Hour Day-Ahead Electricity Price Forecasting using LEAR Model

This script performs a two-step 48-hour rolling forecast for electricity prices 
on the Nord Pool market using the LEAR model. It includes probabilistic post-processing 
and error metrics for each lead time.

Dependencies
------------
- pandas
- numpy
- seaborn
- matplotlib
- epftoolbox (for LEAR model and data handling)
"""

import pandas as pd
import numpy as np
import argparse
import os
import seaborn as sns
import matplotlib.pyplot as plt

from epftoolbox.data import read_data
from epftoolbox.evaluation import MAE, sMAPE
from epftoolbox.models import LEAR

# --------------------------------------------------------------------
# PARAMETERS AND DATA LOADING
# --------------------------------------------------------------------

# Dataset and model settings
dataset = 'NP'  # Nord Pool region
calibration_window = 364 * 2  # Two-year training window

# Testing period for backtesting
begin_test_date = "01/01/2018 00:00"
end_test_date = "31/03/2018 23:00"

# Paths for data and forecasts
path_datasets_folder = os.path.join('.', 'datasets')
path_recalibration_folder = os.path.join('.' , 'experimental_files')

# Load pre-split train/test data from epftoolbox
df_train, df_test = read_data(dataset=dataset,
                              path=path_datasets_folder,
                              begin_test_date=begin_test_date,
                              end_test_date=end_test_date)

# --------------------------------------------------------------------
# REAL VALUE EXTRACTION FOR VERIFICATION
# --------------------------------------------------------------------

# Initialize placeholder for 48h rolling forecasts and real data
forecast = pd.DataFrame(index=df_test.index[::24], columns=['h' + str(k) for k in range(48)])
real_values = []
forecast_index = []

# Manually slice 48-hour real price windows every 24 hours
for i in range(0, len(df_test) - 47, 24):
    window = df_test['Price'].iloc[i:i+48].values
    if len(window) == 48:
        real_values.append(window)
        forecast_index.append(df_test.index[i])

real_values = pd.DataFrame(real_values, index=forecast_index, columns=['h' + str(k) for k in range(48)])
forecast_dates = forecast.index

# --------------------------------------------------------------------
# STEP 1: FORECAST HOURS 0–23 (DAY +1)
# --------------------------------------------------------------------

# Initialize model and forecast DataFrame
model = LEAR(calibration_window=calibration_window)
forecast_d1 = pd.DataFrame(index=forecast_dates, columns=[f'h{k}' for k in range(24)])

for date in forecast_dates:
    # Prepare input: combine train and up-to-date test data
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=23)]])
    
    # Mask the next 24h (to be predicted)
    data_available.loc[date:date + pd.Timedelta(hours=23), 'Price'] = np.NaN

    # Forecast next 24 hours using LEAR
    Yp_d1 = model.recalibrate_and_forecast_next_day(df=data_available,
                                                    next_day_date=date,
                                                    calibration_window=calibration_window)
    forecast_d1.loc[date] = Yp_d1.flatten()

    # Save interim results
    forecast_d1.to_csv(path_recalibration_folder + '/forecast_d1.csv')


# --------------------------------------------------------------------
# STEP 2: FORECAST HOURS 24–47 (DAY +2) USING STEP 1 RESULTS
# --------------------------------------------------------------------

forecast_d1 = pd.read_csv(path_recalibration_folder + '/forecast_d1.csv', index_col=0, parse_dates=True)
forecast_d2 = pd.DataFrame(index=forecast_dates, columns=[f'h{k}' for k in range(24)])
forecast_48h = pd.DataFrame(index=forecast_dates, columns=[f'h{k}' for k in range(48)])

for date in forecast_dates:
    data_available = pd.concat([df_train, df_test.loc[:date + pd.Timedelta(hours=47)]])
    d1_index = pd.date_range(date, date + pd.Timedelta(hours=23), freq='h')

    # Replace day +1 prices with step-1 forecasts
    data_available.loc[d1_index, 'Price'] = forecast_d1.loc[date].values

    # Extend historical window to ensure context for D+2
    df_test_expanded = data_available.loc[date - pd.Timedelta(days=10):date + pd.Timedelta(days=2)]

    try:
        _, _, X_d2 = model._build_and_split_XYs(df_train=data_available,
                                                df_test=df_test_expanded,
                                                date_test=date + pd.Timedelta(days=1))
        Yp_d2 = model.predict(X_d2)
        forecast_d2.loc[date] = Yp_d2.flatten()

        # Concatenate 48h forecast
        forecast_48h.loc[date] = np.concatenate([forecast_d1.loc[date].values, Yp_d2.flatten()])

    except Exception as e:
        print(f"Skipping {date} due to: {e}")
        continue

forecast_48h.to_csv(path_recalibration_folder + '/forecast_48h.csv')


# --------------------------------------------------------------------
# FORECAST TO LONG FORMAT FOR ERROR ANALYSIS
# --------------------------------------------------------------------

df_long = forecast_48h.copy()
df_long.index.name = 'forecast_issue_time'  # Ensure index has the correct name
df_long = df_long.reset_index()  # Now this becomes a column

records = []
for _, row in df_long.iterrows():
    issue_time = row['forecast_issue_time']
    for h in range(48):
        records.append({
            'forecast_issue_time': issue_time,
            'target_time': issue_time + pd.Timedelta(hours=h),
            'lead_time_hr': h,
            'forecast_price': row[f'h{h}']
        })

df_leadtime = pd.DataFrame(records)

# Add actual prices to compute errors
actual_prices = df_test['Price'].copy().reset_index()
actual_prices.rename(columns={'Date': 'target_time', 'Price': 'actual_price'}, inplace=True)
df_leadtime = df_leadtime.merge(actual_prices, on='target_time', how='left')


# --------------------------------------------------------------------
# ERROR METRICS
# --------------------------------------------------------------------

# Clean up
df_leadtime['forecast_price'] = pd.to_numeric(df_leadtime['forecast_price'], errors='coerce')
df_leadtime['abs_error'] = (df_leadtime['forecast_price'] - df_leadtime['actual_price']).abs()
df_leadtime['sMAPE'] = 200 * df_leadtime['abs_error'] / (
    df_leadtime['forecast_price'].abs() + df_leadtime['actual_price'].abs()
)


# Average error by forecast lead time
error_by_lead = df_leadtime.groupby('lead_time_hr')[['abs_error', 'sMAPE']].mean()
print(error_by_lead)

# --------------------------------------------------------------------
# PROBABILISTIC FORECASTING (QUANTILES & SCORING)
# --------------------------------------------------------------------

# Compute rolling residuals
df_leadtime['residual'] = df_leadtime['actual_price'] - df_leadtime['forecast_price']
start_prob_date = pd.Timestamp("2018-03-01")
rolling_window_days = 60
quantile_records = []
forecast_dates = sorted(df_leadtime['forecast_issue_time'].unique())

# Rolling quantile computation per lead time
for current_date in forecast_dates:
    if current_date < start_prob_date:
        continue
    window_start = current_date - pd.Timedelta(days=rolling_window_days)
    residuals_window = df_leadtime[
        (df_leadtime['forecast_issue_time'] >= window_start) &
        (df_leadtime['forecast_issue_time'] < current_date)
    ]

    quantiles = (
        residuals_window
        .groupby('lead_time_hr')['residual']
        .apply(lambda x: x.quantile([0.1, 0.9]) if len(x) >= 30 else pd.Series([np.nan, np.nan], index=[0.1, 0.9]))
        .unstack()
    )
    quantiles.columns = ['q10', 'q90']

    current_forecast = df_leadtime[df_leadtime['forecast_issue_time'] == current_date].copy()
    current_with_quantiles = current_forecast.merge(quantiles, on='lead_time_hr', how='left')
    current_with_quantiles['P10'] = current_with_quantiles['forecast_price'] + current_with_quantiles['q10']
    current_with_quantiles['P90'] = current_with_quantiles['forecast_price'] + current_with_quantiles['q90']
    quantile_records.append(current_with_quantiles)

df_prob_forecast = pd.concat(quantile_records, ignore_index=True)

# --------------------------------------------------------------------
# INTERVAL SCORE & PINBALL LOSS
# --------------------------------------------------------------------

def interval_score(upper, lower, actual, alpha=0.2):
    """
    Compute the interval score for prediction intervals.

    Parameters
    ----------
    upper : pd.Series
        Upper prediction bound.
    lower : pd.Series
        Lower prediction bound.
    actual : pd.Series
        True observed values.
    alpha : float
        Confidence level (e.g. 0.2 for 80% interval).

    Returns
    -------
    pd.Series
        Interval score per observation.
    """
    width = upper - lower
    below = (lower - actual) * (actual < lower)
    above = (actual - upper) * (actual > upper)
    penalty = (2 / alpha) * (below + above)
    return width + penalty

df_prob_forecast['IS_80'] = interval_score(
    df_prob_forecast['P90'],
    df_prob_forecast['P10'],
    df_prob_forecast['actual_price'],
    alpha=0.2
)

def pinball_loss(q, y, tau):
    """
    Compute the pinball loss for quantile forecasts.

    Parameters
    ----------
    q : pd.Series
        Forecasted quantile value.
    y : pd.Series
        Actual observed value.
    tau : float
        Quantile level (e.g. 0.1 or 0.9)

    Returns
    -------
    float
        Pinball loss value.
    """
    return (tau - (y < q)) * (y - q)

df_prob_forecast['PL_P10'] = df_prob_forecast.apply(
    lambda row: pinball_loss(row['P10'], row['actual_price'], 0.1), axis=1
)
df_prob_forecast['PL_P90'] = df_prob_forecast.apply(
    lambda row: pinball_loss(row['P90'], row['actual_price'], 0.9), axis=1
)

# Print average scores
print( "Average Interval Score (80%):", df_prob_forecast['IS_80'].mean())
print("Average Pinball Loss P10:", df_prob_forecast['PL_P10'].mean())
print("Average Pinball Loss P90:", df_prob_forecast['PL_P90'].mean())
